A cute little demo showing the simplest usage of minGPT. Configured to run fine on Macbook Air in like a minute.

In [7]:
import torch
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from mingpt.utils import set_seed

set_seed(3407)

In [22]:
import random

def random_add_instance(length):
    a = [random.randint(0,9) for i in range(length)]
    b = [random.randint(0,9) for i in range(length)]
    val_a = int(''.join(str(d) for d in a))
    val_b = int(''.join(str(d) for d in b))
    val_c = val_a + val_b
    print(a, b, val_a, val_b, val_c)
    str_c = str(val_c)
    str_c = (length + 1 - len(str_c)) * '0' + str_c
    return a + b + [int(d) for d in str_c]

random_add_instance(4)


[9, 3, 3, 4] [6, 8, 6, 1] 9334 6861 16195


[9, 3, 3, 4, 6, 8, 6, 1, 1, 6, 1, 9, 5]

In [10]:
class AddDataset(Dataset):
    """ 
    Dataset for the Add problem. E.g. for problem length 3:
    12 + 333 = 345
    Input: 0 1 2 3 3 3 -> Output: 0 3 4 5
    Which will feed into the transformer concatenated as:
    input:  0 1 2 3 3 3 0 3 4
    output: I I I I I 0 3 4 5
    where I is "ignore", as the transformer is reading the input sequence
    """

    def __init__(self, split, length=3):
        assert split in {'train', 'test'}
        self.split = split
        self.length = length
    
    def __len__(self):
        return 10000 # ...
    
    def get_vocab_size(self):
        return 10
    
    def get_block_size(self):
        # the length of the sequence that will feed into transformer, 
        # containing concatenated input and the output, but -1 because
        # the transformer starts making predictions at the last input element
        return 3 * self.length # + 1 - 1

    def __getitem__(self, idx):
        while True:
            rai = random_add_instance(self.length)
            h = hash(str(rai[:2*self.length]))
            
            inp_split = 'test' if h % 4 == 0 else 'train' # designate 25% of examples as test
            if inp_split == self.split:
                break # ok
        
        x = torch.tensor(rai[:-1], dtype=torch.long)
        y = torch.tensor(rai[1:], dtype=torch.long)
        
        # we only want to predict at output locations, mask out the loss at the input locations
        y[:2*self.length-1] = -1
        return x, y

In [11]:
# print an example instance of the dataset
train_dataset = AddDataset('train')
test_dataset = AddDataset('test')
x, y = train_dataset[0]

print (x)
for a, b in zip(x,y):
    print(int(a),int(b))

tensor([2, 9, 3, 4, 8, 0, 0, 7, 7])
2 -1
9 -1
3 -1
4 -1
8 -1
0 0
0 7
7 7
7 3


In [12]:
# create a GPT instance
from mingpt.model import GPT

model_config = GPT.get_default_config()
model_config.model_type = 'gpt-micro'
#model_config.model_type = 'gpt-nano'

model_config.vocab_size = train_dataset.get_vocab_size()
model_config.block_size = train_dataset.get_block_size()
model = GPT(model_config)

number of parameters: 0.80M


In [13]:
print (model_config.n_head, model_config.n_layer, model_config.n_embd)

4 4 128


In [14]:
# create a Trainer object
from mingpt.trainer import Trainer

train_config = Trainer.get_default_config()
train_config.learning_rate = 5e-4 # the model we're using is so small that we can go a bit faster
train_config.max_iters = 5000
train_config.num_workers = 0
trainer = Trainer(train_config, model, train_dataset)

running on device cpu


In [15]:
def batch_end_callback(trainer):
    if trainer.iter_num % 100 == 0:
        print(f"iter_dt {trainer.iter_dt * 1000:.2f}ms; iter {trainer.iter_num}: train loss {trainer.loss.item():.5f}")
trainer.set_callback('on_batch_end', batch_end_callback)

trainer.run()

/Users/janek/dev/uwr/language-models/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


iter_dt 0.00ms; iter 0: train loss 2.32162
iter_dt 35.08ms; iter 100: train loss 1.54925
iter_dt 34.68ms; iter 200: train loss 1.52188
iter_dt 35.08ms; iter 300: train loss 1.38872
iter_dt 34.88ms; iter 400: train loss 1.27035
iter_dt 35.81ms; iter 500: train loss 0.76410
iter_dt 36.88ms; iter 600: train loss 0.65160
iter_dt 35.41ms; iter 700: train loss 0.57840
iter_dt 36.64ms; iter 800: train loss 0.54774
iter_dt 35.39ms; iter 900: train loss 0.48151
iter_dt 36.67ms; iter 1000: train loss 0.45590
iter_dt 37.72ms; iter 1100: train loss 0.38573
iter_dt 36.88ms; iter 1200: train loss 0.46371
iter_dt 36.11ms; iter 1300: train loss 0.49345
iter_dt 36.25ms; iter 1400: train loss 0.37969
iter_dt 35.36ms; iter 1500: train loss 0.39450
iter_dt 38.59ms; iter 1600: train loss 0.34649
iter_dt 37.19ms; iter 1700: train loss 0.30424
iter_dt 35.73ms; iter 1800: train loss 0.25734
iter_dt 37.35ms; iter 1900: train loss 0.29334
iter_dt 46.19ms; iter 2000: train loss 0.29891
iter_dt 48.36ms; iter 2100

In [1]:
# now let's perform some evaluation
model.eval()
None

NameError: name 'model' is not defined

In [19]:
def eval_add_split(trainer, split, max_batches):
    dataset = {'train':train_dataset, 'test':test_dataset}[split]
    n = train_dataset.length # naugy direct access shrug
    results = []
    mistakes_printed_already = 0
    loader = DataLoader(dataset, batch_size=100, num_workers=0, drop_last=False)
    #loader = DataLoader(dataset, batch_size=1, num_workers=0, drop_last=False)
    for b, (x, y) in enumerate(loader):
        x = x.to(trainer.device)
        y = y.to(trainer.device)

        inp = x[:, :2*n]
        sol = y[:, -n-1:]
        
        cat = model.generate(inp, n+1, do_sample=False) # using greedy argmax, not sampling
        sol_candidate = cat[:, -n-1:]         
        correct = (sol == sol_candidate).all(1).cpu() 
        for i in range(x.size(0)):
            results.append(int(correct[i]))
    
    rt = torch.tensor(results, dtype=torch.float)
    print("%s final score: %d/%d = %.2f%% correct" % (split, rt.sum(), len(results), 100*rt.mean()))
    return rt.sum()

# run a lot of examples from both train and test through the model and verify the output correctness
with torch.no_grad():
    train_score = eval_add_split(trainer, 'train', max_batches=50)
    test_score  = eval_add_split(trainer, 'test',  max_batches=50)

train final score: 9690/10000 = 96.90% correct
test final score: 9712/10000 = 97.12% correct
